In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import nltk

from tqdm import trange
from nltk import tokenize
from nltk.corpus import stopwords   
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from underthesea import word_tokenize

In [12]:
import warnings
warnings.filterwarnings('ignore')
nltk.download('omw-1.4', quiet=True)
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (17,7)
plt.rcParams['font.size'] = 18

# Loading the Data

In [65]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

data = pd.read_csv("Dataset/vsa_food_rv_train.csv", encoding="utf-8")
    
data.head(10)

,Comment,Rating
0,Món ăn cực tệ,0.0
1,Mình thấy mọi người review tích cực nhưng thực sự mình không thấy lẩu ở đây giống review. Lẩu đắt và chán thực sự. Never come back!,0.0
2,"Chắc với sinh viên thì ai cũng quen thuộc với lẩu Nguyệt. Cứ có dịp gì tụ tập đông đông là nhóm mình lại qua đây ăn. Vừa ngon vừa rẻ. Quán thì phải kéo dài từ vỉa hè đến cả trong ngõ 106, có cả ngồi dưới nhà và đi thang máy lên tầng tùy mình muốn chọn ngồi đâu thì ngồi.\n Quán cũng chỉ có 2 loại lẩu: lẩu riêu cua và lẩu hải sản. Nhân viên thường lấy đồ theo kiểu 100k/ người. Lẩu riêu cua nếu đi 7 người thì cũng chỉ gọi phần cho 5 người thôi, không sẽ không hết vì quá nhiều )). Thường bọn mình sẽ gọi thêm 1 đĩa lòng, tràng, trứng gà non 100k/ đĩa ăn kèm. Còn lẩu hải sản cũng khá đầy đặn 350k/ nồi. Quán cũng bán thêm quẩy ăn kèm lẩu nữa. Nói chung ăn ở đây cứ chia đầu người 120k là no lắm rồi ý ^^.",1.0
3,"Nhiều hôm thèm lẩu riêu mà chỉ có một mình thì nên ra đây để ăn vì giá rổ rất phải chăng để có thể không cần đồng bọn.\nTết nhất ngán thịt thì có thể qua đây vì lẩu ở đây cực nhiều đậu. Thuộc thể loại ăn lẩu thích ăn rau nhiều hơn thịt nên quán này cực hợp với mình. Gọi nồi lẩu 2 người không thêm gì khác cũng đủ no căng bụng rồi. Topping ở đây còn có cả chả cá, chả bò, giò tai cực ngon. Ăn một người cũng hết 200k thui à.",1.0
4,Không gian sạch sẽ.quán mới lên ra món hơi chậm.15p chờ cũng ko lâu lắm.nhưng đồ đều nóng thơm lừng.nhiều nữa chứ.,1.0
5,Nhất định sẽ ủng hộ quán dài dài…,1.0
6,Lang thang với cái bụng đói.đứng chờ tàu thì ngó thấy.quay xe đi vào.lúc đầu nghĩ là nhầm quán cafe cơ.vì bàn ghê đẹp.có hai cô chỉ cũng đẹp luôn.đói mạnh dạn gọi hẳn đặc biệt 60k.\nĂn ngon nhiều.còn bán rượu khá ngon.đồ đủ để hai ng ngồi nhấm vài ly luôn.\nNói chung 5 sao,1.0
7,"Cá thái quá to, rán cháy, cứng, khô.\nNước dùng chán, nhạt",0.0
8,"Đói quá vội ăn nên ko chụp đc cái ảnh tử tế. Phải nói là bún cá ở đây ngon nhất trong các quán đã ăn. \nMình ăn thử rất rất nhiều quán, cả các quán nổi tiếng lẫn các quán nhỏ nhưng ko ở đâu làm ổn như ở đây. \nNước dùng đậm đà, màu sắc bắt mắt chứ ko trắng nhởn như nhiều quán khác. Cá thì to bự cắn đã cái mồm, giòn tan luôn nhưng ko hề bị khô. Theo thói quen mình mua thêm cá vì các quán khác cho cá rất ít. Còn ở đây cá của 1 suất thôi cũng nhiều hơn hẳn các quán khác, thực sự to và nhiều cực luôn. Có vài miếng cá nhiều xương nhưng chấp nhận đc vì ngon. Nước chấm cũng ngon khỏi bàn. Đồ kèm theo chuẩn bị đầy đủ.\nThái độ bán hàng cũng tốt. Nếu là quán khác thì cứ tự ý mà cho theo ý mình thôi. Còn ở đây gọi lịch sự hỏi khách ăn rau cần đc ko vì tạm thời ko có cải. Vậy mà khi nhận đồ vẫn đc cả 2 suất rau cải. \nĐến anh chồng Nhật khó tính cũng phải khen là tốt nhất từ trc đến nay.\nHy vọng lần sau ăn sẽ ko có gì thay đổi.",1.0
9,"Mình vẫn k hiểu vì sao quán mở được đến tận 7-8 cơ sở tại HN và SG )))) Đồ ăn ở mức trung bình, nước lẩu thái chả có gì đặc sắc. Nồi lẩu nhỏ hơn 300k ăn lèo tèo vài miếng thịt, tôm bở ko tươi, nấm cho 2 người ăn được 1 nhúm. Món cánh gà tre ăn tạm ổn nhất, xôi xoài ăn rõ chán, xoài nhũn và nhạt nhẽo. Giá quá đắt so với những gì đã được trải nghiệm.",0.0


# Xử lý dữ liệu

In [66]:
# Drop NaN

data = data.dropna()
data = data.reset_index(drop=True)
data['Rating'].value_counts()

Rating
1.0    17351
0.0     5490
Name: count, dtype: int64

In [67]:
# Load teencode map

teencode_file = "Dataset/teencode.txt"

vi_teencode_map = {}
cnt = 0
with open(teencode_file, "r", encoding="utf-16") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue

        parts = line.split("\t")
        if len(parts) == 2:
            key, value = parts
            vi_teencode_map[key.strip()] = value.strip()

print("Số từ teencode: ", len(vi_teencode_map))
print(dict(list(vi_teencode_map.items())[:10]))

Số từ teencode:  460
{'a': 'anh', 'acc': 'account', 'ace': 'anh chị em', 'ad': 'admin', 'ae': 'anh em', 'ah': 'à', 'àh': 'à', 'ạh': 'ạ', 'ahihi': 'hi hi', 'aiu': 'anh yêu'}


In [68]:
# Load icon map
emoji_file = "Dataset/emoji_dict.txt"

emoji_map = {}
with open(emoji_file, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue

        parts = line.split("\t")
        if len(parts) == 2:
            key, value = parts
            emoji_map[key.strip()] = value.strip()

print("Số emoji: ", len(emoji_map))
print(dict(list(emoji_map.items())[:10]))

Số emoji:  3521
{'😀': 'emj_grinning_face', '😃': 'emj_grinning_face_with_big_eyes', '😄': 'emj_grinning_face_with_smiling_eyes', '😁': 'emj_beaming_face_with_smiling_eyes', '😆': 'emj_grinning_squinting_face', '😅': 'emj_grinning_face_with_sweat', '🤣': 'emj_rolling_on_the_floor_laughing', '😂': 'emj_face_with_tears_of_joy', '🙂': 'emj_slightly_smiling_face', '🙃': 'emj_upside_down_face'}


In [81]:
# Load unit map

unit_file = "Dataset/unit_dict.txt"

unit_map = {}
with open(unit_file, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue

        parts = line.split("\t")
        if len(parts) == 2:
            key, value = parts
            unit_map[key.strip()] = value.strip()

print("Số unit: ", len(unit_map))
print(dict(list(unit_map.items())[:10]))

Số unit:  11
{'p': 'phút', 's': 'giây', 'l': 'lít', 'g': 'gam', 'kg': 'ki_lo_gam', 'h': 'giờ', 'k': 'ngàn', 'M': 'triệu', 'B': 'tỷ', 'm': 'mét'}


In [101]:
def remove_lengthening(word):
    return re.sub(r'(.)\1{2,}', r'\1\1', word)


def normalize_punct_spacing(text):
    # Thêm space trước dấu câu đơn (không áp dụng khi chuỗi dấu)
    text = re.sub(r'(?<![\s\.\,\!\?\:\;\)\(])([.,!?:;])', r' \1', text)

    # Thêm space sau dấu câu đơn
    text = re.sub(r'([.,!?:;])(?![\s\.\,\!\?\:\;\)\(])', r'\1 ', text)

    # Loại bỏ space dư
    text = re.sub(r'\s+', ' ', text).strip()

    return text


def normalize_number_units(text, unit_map):
    def repl(m):
        num = m.group(1)
        suf = m.group(2).lower()
        punct = m.group(3) or ""
        
        if suf in unit_map:   
            return f"{num} {unit_map[suf]}{punct}"
        else:
            return m.group(0) 
    
    unit_pattern = "|".join(sorted(unit_map.keys(), key=lambda x: -len(x)))
    pattern = rf"(\d+)({unit_pattern})([,.!?:]?)"
    
    return re.sub(pattern, repl, text, flags=re.IGNORECASE)


def replace_teencode_and_emoji(words, teencode_map, emoji_map):
    new_words = []
    
    for w in words:
        if w in teencode_map:
            new_words.append(teencode_map[w])
        elif w in emoji_map:
            new_words.append(emoji_map[w])
        else:
            new_words.append(w)
            
    return new_words


def clean_vi(text):
    text = str(text)
    text = text.replace('\n', ' ')
    text = text.lower()
    text = re.sub(r'\.(\S)', r'. \1', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = normalize_number_units(text, unit_map)
    text = normalize_punct_spacing(text)
    
    #Split bằng khoảng trắng, không dùng tokenizer của underthesea (do nó bẩn, kết quả tệ, ghép từa lưa)
    words = text.split()

    words = [remove_lengthening(w) for w in words]
    words = replace_teencode_and_emoji(words, vi_teencode_map, emoji_map)

    clean_words = [w for w in words]
    return " ".join(clean_words)


In [119]:
df = data.copy()
df['Comment'] = df['Comment'].apply(clean_vi)
df['word_count'] = df['Comment'].apply(lambda x: len(x.split()))

df.head(15)

,Comment,Rating,word_count
0,món ăn cực tệ,0.0,4
1,mình thấy mọi người review tích cực nhưng thực sự mình không thấy lẩu ở đây giống review . lẩu đắt và chán thực sự . never come back !,0.0,30
2,"chắc với sinh viên thì ai cũng quen thuộc với lẩu nguyệt . cứ có dịp gì tụ tập đông đông là nhóm mình lại qua đây ăn . vừa ngon vừa rẻ . quán thì phải kéo dài từ vỉa hè đến cả trong ngõ 106 , có cả ngồi dưới nhà và đi thang máy lên tầng tùy mình muốn chọn ngồi đâu thì ngồi . quán cũng chỉ có 2 loại lẩu : lẩu riêu cua và lẩu hải sản . nhân viên thường lấy đồ theo kiểu 100 ngàn/ người . lẩu riêu cua nếu đi 7 người thì cũng chỉ gọi phần cho 5 người thôi , không sẽ không hết vì quá nhiều )). thường bọn mình sẽ gọi thêm 1 đĩa lòng , tràng , trứng gà non 100 ngàn/ đĩa ăn kèm . còn lẩu hải sản cũng khá đầy đặn 350 ngàn/ nồi . quán cũng bán thêm quẩy ăn kèm lẩu nữa . nói chung ăn ở đây cứ chia đầu người 120 ngàn là no lắm rồi ý ^^ .",1.0,181
3,"nhiều hôm thèm lẩu riêu mà chỉ có một mình thì nên ra đây để ăn vì giá rổ rất phải chăng để có thể không cần đồng bọn . tết nhất ngán thịt thì có thể qua đây vì lẩu ở đây cực nhiều đậu . thuộc thể loại ăn lẩu thích ăn rau nhiều hơn thịt nên quán này cực hợp với mình . gọi nồi lẩu 2 người không thêm gì khác cũng đủ no căng bụng rồi . topping ở đây còn có cả chả cá , chả bò , giò tai cực ngon . ăn một người cũng hết 200 ngàn thôi à .",1.0,109
4,không gian sạch sẽ . quán mới lên ra món hơi chậm . 15 phút chờ cũng không lâu lắm . nhưng đồ đều nóng thơm lừng . nhiều nữa chứ .,1.0,32
5,nhất định sẽ ủng hộ quán dài dài…,1.0,8
6,lang thang với cái bụng đói . đứng chờ tàu thì ngó thấy . quay xe đi vào . lúc đầu nghĩ là nhầm quán cafe cơ . vì bàn ghê đẹp . có hai cô chỉ cũng đẹp luôn . đói mạnh dạn gọi hẳn đặc biệt 60 ngàn . ăn ngon nhiều . còn bán rượu khá ngon . đồ đủ để hai người ngồi nhấm vài ly luôn . nói chung 5 sao,1.0,76
7,"cá thái quá to , rán cháy , cứng , khô . nước dùng chán , nhạt",0.0,17
8,"đói quá vội ăn nên không chụp được cái ảnh tử tế . phải nói là bún cá ở đây ngon nhất trong các quán đã ăn . mình ăn thử rất rất nhiều quán , cả các quán nổi tiếng lẫn các quán nhỏ nhưng không ở đâu làm ổn như ở đây . nước dùng đậm đà , màu sắc bắt mắt chứ không trắng nhởn như nhiều quán khác . cá thì to bự cắn đã cái mồm , giòn tan luôn nhưng không hề bị khô . theo thói quen mình mua thêm cá vì các quán khác cho cá rất ít . còn ở đây cá của 1 suất thôi cũng nhiều hơn hẳn các quán khác , thực sự to và nhiều cực luôn . có vài miếng cá nhiều xương nhưng chấp nhận được vì ngon . nước chấm cũng ngon khỏi bàn . đồ kèm theo chuẩn bị đầy đủ . thái độ bán hàng cũng tốt . nếu là quán khác thì cứ tự ý mà cho theo ý mình thôi . còn ở đây gọi lịch sự hỏi khách ăn rau cần được không vì tạm thời không có cải . vậy mà khi nhận đồ vẫn được cả 2 suất rau cải . đến anh chồng nhật khó tính cũng phải khen là tốt nhất từ trước đến nay . hy vọng lần sau ăn sẽ không có gì thay đổi .",1.0,243
9,"mình vẫn không hiểu vì sao quán mở được đến tận 7-8 cơ sở tại hà nội và sài gòn )) đồ ăn ở mức trung bình , nước lẩu thái chả có gì đặc sắc . nồi lẩu nhỏ hơn 300 ngàn ăn lèo tèo vài miếng thịt , tôm bở không tươi , nấm cho 2 người ăn được 1 nhúm . món cánh gà tre ăn tạm ổn nhất , xôi xoài ăn rõ chán , xoài nhũn và nhạt nhẽo . giá quá đắt so với những gì đã được trải nghiệm .",0.0,97


# Lưu dữ liệu

In [120]:
output_path = "Dataset/vsa_food_rv_train_clean.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print("Đã lưu file TRAIN cleaned CSV tại:", output_path)

Đã lưu file TRAIN cleaned CSV tại: Dataset/vsa_food_rv_train_clean.csv


In [121]:
test = pd.read_csv("Dataset/vsa_food_rv_test.csv", encoding="utf-8")

test = test.dropna().reset_index(drop=True)
test['Comment'] = test['Comment'].apply(clean_vi)
test['word_count'] = test['Comment'].apply(lambda x: len(x.split()))

output_path_test = "Dataset/vsa_food_rv_test_clean.csv"
test.to_csv(output_path_test, index=False, encoding="utf-8-sig")

print("Đã lưu file TEST cleaned:", output_path_test)

Đã lưu file TEST cleaned: Dataset/vsa_food_rv_test_clean.csv
